<a href="https://colab.research.google.com/github/arinajman/personal/blob/main/Project3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Obtener datos del US Treasury Bill a 1 año

In [1]:
!pip install fredapi

In [2]:
import pandas as pd
from pandas_datareader import data as pdr
import datetime
from fredapi import Fred
import os

In [3]:
#help(os)
#dir(os)
#help(os.environ)

In [14]:
# Establecer la variable de entorno (solo para la sesión actual)
os.environ['API_KEY'] = '' #'TU_CLAVE_SECRETA'
# Acceder a la variable de entorno en tu código
api_key = os.environ.get('API_KEY')

if api_key:
    print(f"La clave API es: {api_key[:5]}...") # Muestra solo los primeros caracteres por seguridad
    # Utiliza api_key en tus llamadas a la API
else:
    print("La clave API no está configurada como variable de entorno.")

La clave API no está configurada como variable de entorno.


In [5]:
#help(pdr)
#dir(pdr)
#help(pdr.DataReader)
#dir(pdr.DataReader)

In [6]:
#help(Fred)
#dir(Fred)

In [7]:
fred=Fred(api_key)
fred

In [8]:
ticker = 'dgs1' #DGS1 is the 1-year US Treasury Bill

In [9]:
fred.get_series_info(ticker)

,0
id,DGS1
realtime_start,2025-04-28
realtime_end,2025-04-28
title,Market Yield on U.S. Treasury Securities at 1-...
observation_start,1962-01-02
observation_end,2025-04-25
frequency,Daily
frequency_short,D
units,Percent
units_short,%


In [10]:
print('observation_start:',fred.get_series_info(ticker)['observation_start'],type(fred.get_series_info(ticker)['observation_start']))
year_start = int(fred.get_series_info(ticker)['observation_start'][:4])
print('year_start:',year_start,type(year_start))
month_start = int(fred.get_series_info(ticker)['observation_start'][5:7])
print('month_start:',month_start,type(month_start))
day_start = int(fred.get_series_info(ticker)['observation_start'][8:10])
print('day_start:',day_start,type(day_start))
start=datetime.datetime(year_start,month_start,day_start)
print('start:',start,type(start))

observation_start: 1962-01-02 <class 'str'>
year_start: 1962 <class 'int'>
month_start: 1 <class 'int'>
day_start: 2 <class 'int'>
start: 1962-01-02 00:00:00 <class 'datetime.datetime'>


In [11]:
df = pdr.DataReader(ticker, 'fred',start)
df

,dgs1
DATE,
1962-01-02,3.22
1962-01-03,3.24
1962-01-04,3.24
1962-01-05,3.26
1962-01-08,3.31
...,...
2025-04-21,3.95
2025-04-22,3.98
2025-04-23,4.01


In [12]:
df_final = df.groupby(df.index.year).last()
df_final = df_final.iloc[:-1]
df_final

,dgs1
DATE,
1962,3.05
1963,3.83
1964,3.99
1965,4.96
1966,5.00
...,...
2020,0.10
2021,0.39
2022,4.73


In [13]:
negative_count = len(df_final[df_final[ticker] < 0])
between_0_1_count = len(df_final[(df_final[ticker] >= 0) & (df_final[ticker] <= 1)])
between_1_3_count = len(df_final[(df_final[ticker] > 1) & (df_final[ticker] <= 3)])
between_3_5_count = len(df_final[(df_final[ticker] > 3) & (df_final[ticker] <= 5)])
between_5_7_count = len(df_final[(df_final[ticker] > 5) & (df_final[ticker] <= 7)])
greater_than_7_count = len(df_final[df_final[ticker] > 7])

print(f"Cantidad de años negativos: {negative_count}")
print(f"Cantidad entre 0 y 1: {between_0_1_count}")
print(f"Cantidad entre 1 y 3: {between_1_3_count}")
print(f"Cantidad entre 3 y 5: {between_3_5_count}")
print(f"Cantidad entre 5 y 7: {between_5_7_count}")
print(f"Cantidad mayor a 7: {greater_than_7_count}")

Cantidad de años negativos: 0
Cantidad entre 0 y 1: 11
Cantidad entre 1 y 3: 7
Cantidad entre 3 y 5: 18
Cantidad entre 5 y 7: 12
Cantidad mayor a 7: 15
